# $\text{LiNiPO}_4$ System

Adapted from: https://sunnysuite.github.io/Sunny.jl/stable/examples/spinw/SW13_LiNiPO4.html

### Crystal Cell

In [34]:
using Sunny, GLMakie

units = Units(:meV, :angstrom);

The unit cell has the following lattice vectors


|     |  a [Å]  |  b [Å]  |  c [Å]  |
| --- | --- | --- | --- |
| LiNiPO4 | 10.02 | 5.86 | 4.68 | 

In [35]:
latvecs = lattice_vectors(10.02, 5.86, 4.68, 90, 90, 90)

3×3 StaticArraysCore.SMatrix{3, 3, Float64, 9} with indices SOneTo(3)×SOneTo(3):
 10.02  0.0   0.0
  0.0   5.86  0.0
  0.0   0.0   4.68

In [36]:
cryst = Crystal(latvecs, [[1/4, 1/4, 0]], 62, types = ["Ni"])
# QUESTION: Spacegroup 62?

Crystal
Spacegroup 'P n m a' (62)
Lattice params a=10.02, b=5.86, c=4.68, α=90°, β=90°, γ=90°
Cell volume 274.8
Type 'Ni', Wyckoff 4c (site sym. '.m.'):
   1. [1/4, 1/4, 0]
   2. [3/4, 3/4, 0]
   3. [3/4, 1/4, 1/2]
   4. [1/4, 3/4, 1/2]


In [37]:
view_crystal(cryst)

### Spin System

In [38]:
sys = System(cryst, [1 => Moment(s=1, g=2)], :dipole_uncorrected)
# QUESTION: Why dipole_uncorrected?
# ANSWER: "The mode :dipole_uncorrected avoids a classical-to-quantum rescaling factor 
# of anisotropy strengths, as needed for consistency with the original fits [of the data].

System [Dipole mode, large-s]
Supercell (1×1×1)×4
Energy per site 0


The interaction interaction energies need to be set

In [39]:
Jbc =  1.036
Jb  =  0.6701
Jc  = -0.0469
Jac = -0.1121
Jab =  0.2977
Da  =  0.1969
Db  =  0.9097
Dc  =  0.0
set_exchange!(sys, Jbc, Bond(2, 3, [0, 0, 0]))
set_exchange!(sys, Jc , Bond(1, 1, [0, 0,-1]))
set_exchange!(sys, Jb , Bond(1, 1, [0, 1, 0]))
set_exchange!(sys, Jab, Bond(1, 2, [0, 0, 0]))
set_exchange!(sys, Jab, Bond(3, 4, [0, 0, 0]))
set_exchange!(sys, Jac, Bond(3, 1, [0, 0, 0]))
set_exchange!(sys, Jac, Bond(4, 2, [0, 0, 0]))
set_onsite_coupling!(sys, S -> Da*S[1]^2 + Db*S[2]^2 + Dc*S[3]^2, 1)

# QUESTION: What about the missing 3 interactions?

In [40]:
view_crystal(sys)

### Optimizing spins
We can now minimize the energy

In [41]:
randomize_spins!(sys)
minimize_energy!(sys)
energy_per_site(sys)

-2.2683999999999997

... and plot the spins in the ground state found above

In [42]:
plot_spins(sys; color=[S[3] for S in sys.dipoles])

### Spin wave theory

In [43]:
swt = SpinWaveTheory(sys; measure=ssf_perp(sys))

SpinWaveTheory [Dipole mode, large-s]
  4 atoms


We define a path connecting high symmetry points in the reciprocal lattice. 600 points are sampled along this path

In [44]:
qpaths = [
	q_space_path(cryst, [[ 0,1,0], [2,1,0]], 600),
	q_space_path(cryst, [[ 0,0,0], [0,2,0]], 600),
	q_space_path(cryst, [[ 0,1,0], [0,1,2]], 600),
]

3-element Vector{Sunny.QPath}:
 Sunny.QPath (600 samples)
 Sunny.QPath (600 samples)
 Sunny.QPath (600 samples)

The intensities can now be found and shown (compare to Jensen et al.)

In [45]:
res = [intensities_bands(swt, qpath, kT=2) for qpath in qpaths]
# Actual measurement temperatures in Jensen et al. (2, 2, 10) K

fig = Figure(size=(1600, 500))

for i in 1:3
	ax = plot_intensities!(fig[1, i], res[i]; units, title="LiNiPO₄ LSWT")
end
fig